## PSTAT 134 Project

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
OgData = pd.read_csv('/home/jovyan/work/Grand-Finale/dataset/movie_metadata_cleaned.csv', header='infer')

In [ ]:
OgData.head()

In [ ]:
#dropping unnecessary columns 
OgData=OgData.drop(['id','imdb_id','status'],axis=1)

In [ ]:
# replacing 0 with np.nan 
OgData[OgData['revenue']==0].shape
OgData['revenue']=OgData['revenue'].replace(0,np.nan)
OgData['budget']=OgData['budget'].replace(0,np.nan)

In [ ]:
type(OgData['genres'])
#OgData['genres']

In [ ]:
#formatting csv columns that are in json/dict format and converting them to lists
# only run once or error
from ast import literal_eval
OgData['genres']=OgData['genres'].fillna('[]').apply(literal_eval).apply(lambda x:[i['name'] for i in x] if isinstance(x,list) else [])
OgData['production_companies']=OgData['production_companies'].fillna('[]').apply(literal_eval).apply(lambda x:[i['name'] for i in x] if isinstance(x,list) else [])
OgData['production_countries']=OgData['production_countries'].fillna('[]').apply(literal_eval).apply(lambda x:[i['name'] for i in x] if isinstance(x,list) else [])
OgData['spoken_languages']=OgData['spoken_languages'].fillna('[]').apply(literal_eval).apply(lambda x:[i['name'] for i in x] if isinstance(x,list) else [])
OgData['belongs_to_collection']=OgData['belongs_to_collection'].fillna('[]').apply(literal_eval).apply(lambda x:x['name'] if isinstance(x,dict) else np.nan)
# Adding in return where >1 is profit and <1 is loss
OgData['return']=OgData['revenue']/OgData['budget']


In [ ]:
OgData

In [ ]:
OgData.poster_path[0]

In [ ]:
BaseUrl = 'http://image.tmdb.org/t/p/w185'

In [ ]:
x=BaseUrl+OgData.poster_path[0]
x

In [ ]:
from skimage import io
x=io.imread(BaseUrl+OgData.poster_path[0])

In [ ]:
io.imshow(x)

In [ ]:
OgData['poster_path'].isnull().sum()

In [ ]:
OgData[OgData['poster_path'].isnull()].index

In [ ]:
# reading in actual pictures
Poster=[]
for i in range(len(OgData)):
    if i==2407 or i==2429:
        Poster.append(np.array([0]))
    else:
        y=io.imread(BaseUrl+OgData.poster_path[i])
        Poster.append(y)
    

In [ ]:
from skimage.io._plugins import matplotlib_plugin

In [ ]:
#preview first 6
import matplotlib.pyplot as plt
for i in range(6):
    plt.imshow(Poster[i],interpolation='nearest',aspect='auto')
    plt.show()

In [ ]:
#append list of image array as column to dataframe
Poster_Image_Array=pd.Series(Poster)
#Note that this code inserts alot of info into the dataframe and slows down
#the ability to access values in OgData. ONly Necessary for Image Analysis
OgData['Poster_Image_Array']=Poster_Image_Array.values

In [ ]:
OgData.head()

In [ ]:
#Number of Movies by Genre
Unique_Genres=[]
for i in OgData['genres']:
    for y in i:
        if y not in Unique_Genres:
            Unique_Genres.append(y)

In [ ]:
Unique_Genres
#print("The Total Number of Genres is %s" %len(Unique_Genres))

In [ ]:
# counting hhow many occurences of each genre
count_genre=[]
for i in Unique_Genres:
    y=sum(x.count(i) for x in OgData['genres'])
    count_genre.append(y)
count_genre
    

        

In [ ]:
#Poster[][2 is RGB Values][Pixel number][0 is red, 1 is blue and 2 is green]
len(Poster[2])

In [ ]:
OgData['Poster_Image_Array'][0]

In [ ]:
#OgData['Poster_Image_Array'][column index]
#[2 to access RGB Info for all pixels]
#[individual pixel RGB value][0 is R, 1 is G, 2 is B]
#185 pixels per image
Average_RGB_Values=[]
for j in range(len(OgData['Poster_Image_Array'])):
    R_Values=[]
    G_Values=[]
    B_Values=[]
    if j==2407 or j==2429:
        Average_RGB_Values.append([0,0,0])
    else:
        for y in range(3):
            for i in range(185):
                if y==0: 
                    R_Values.append(OgData['Poster_Image_Array'][j][2][i][y])
                elif y==1:
                    G_Values.append(OgData['Poster_Image_Array'][j][2][i][y])
                elif y==2:
                    B_Values.append(OgData['Poster_Image_Array'][j][2][i][y])
                
        Average_RGB_Values.append([np.mean(R_Values),np.mean(G_Values),np.mean(B_Values)])
    
            
        

In [ ]:
# Rounding up values
import math as m
for i in range (len(Average_RGB_Values)):
    for y in range(len(Average_RGB_Values[i])):
        Average_RGB_Values[i][y]=m.ceil(Average_RGB_Values[i][y])
        

In [ ]:
 Average_RGB_Values

In [ ]:
# Adding Average_RGB_Values to DataFrame 
m=pd.Series(Average_RGB_Values)
OgData['Avg_RGB_Value']=m.values
OgData.head()
#interesting that the first 3 here ave RGB are [0,0,0] since tehre is a 
#an image produced mostly black....

In [ ]:
OgData.head()

In [ ]:
#Image Analysis
import matplotlib.pyplot as plt
from sklearn import svm,metrics

In [ ]:
#
n_samples=len(Poster)
data=np.array(Poster).reshape((n_samples,-1))


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb=MultiLabelBinarizer()
mlb.fit_transform([i for i in OgData['genres']])
mlb.classes_

In [ ]:
#Adding in Binary Features
OgData2['vote_avg_bin']=np.where(OgData['vote_average']>7,1,0)
OgData2['Movie_Collection_bool']=np.where(OgData['belongs_to_collection'] == np.nan,0,1)
OgData2['Number_Genres']=OgData['genres'].apply(lambda x: len(x)) 
OgData2['Number_Production_Companies']=OgData['production_companies'].apply(lambda x: len(x)) 
OgData2['Number_Production_Countries']=OgData['production_countries'].apply(lambda x: len(x)) 
for i in Unique_Genres:
    OgData2['is_'+str(i)]=OgData['genres'].apply(lambda x:1 if i in x else 0)
            

In [ ]:
OgData2.describe()

In [ ]:
#Random Forest
